In [6]:
import tkinter as tk
from tkinter import messagebox

class ChessBoard:
    def __init__(self, master):
        self.master = master
        self.square_size = 75
        self.master.title("Chess Board GUI")
        self.board = [['' for _ in range(8)] for _ in range(8)]
        self.rectangle_ids = [['' for _ in range(8)] for _ in range(8)]
        self.text_ids= [['' for _ in range(8)] for _ in range(8)]
        self.canvas = tk.Canvas(self.master, width=600, height=600)
        self.canvas.grid()
        self.draw_board()
        self.selected_piece = None  
        self.selected_rectangle = None
        self.prow,self.pcol=None,None
        self.counter=0
        messagebox.showinfo("Welcome!", "White turns first!")
        
    def assign_rectangle_id(self,row,col,rectangle_id):
        self.rectangle_ids[row][col]=rectangle_id
    
    def assign_text_id(self,row,col,text_id):
        self.text_ids[row][col]=text_id
        
    def assign_board(self,row,col,piece):
        self.board[row][col]=piece
    
    def remove_text_id(self,row,col,text_id):
        self.canvas.delete(text_id)
        
    def display_piece(self, row, col):
        text_id = self.text_ids[row][col]
        piece = self.board[row][col]
        self.canvas.itemconfigure(text_id, text=piece)
        
    def is_white(self,piece):
        if piece in ['♖', '♘', '♗', '♕', '♔','♙']:
            return True
        else: return False
        
    def is_black(self,piece):
        if piece in ['♜', '♞', '♝', '♛', '♚','♟']:
            return True
        else: return False
    
    def is_king_under_attack(self, king_piece):
        # Find the coordinates of the king
        king_row, king_col = None, None
        for row in range(8):
            for col in range(8):
                if self.board[row][col] == king_piece:
                    king_row, king_col = row, col
                    break
            if king_row is not None:
                break
        rectangle_id=self.rectangle_ids[king_row][king_col]
        # Check if the king is under attack by any opposing pieces
        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if piece != '' and ((self.is_black(king_piece) and self.is_white(piece)) or (self.is_white(king_piece) and self.is_black(piece))):
                    if self.move(piece, row, col, king_row, king_col):
                        self.canvas.itemconfig(rectangle_id, outline='#e03009', width=2)
                        return True
                
        return False
    
    def move(self, piece, start_row, start_col, dest_row, dest_col):
        # Get the direction of movement (horizontal, vertical, diagonal)
        delta_row = dest_row - start_row
        delta_col = dest_col - start_col
        print( piece, start_row, start_col, dest_row, dest_col,delta_row,delta_col)

        # Check if the destination is within the board boundaries
        if dest_row < 0 or dest_row >= 8 or dest_col < 0 or dest_col >= 8:
            return False

        # Check if the destination is occupied by a piece of the same color
        if self.is_white(piece) and self.is_white(self.board[dest_row][dest_col]):
            return False
        if self.is_black(piece) and self.is_black(self.board[dest_row][dest_col]):
            return False

        # Handle the movement based on the piece type
        if piece in ['♖', '♜']:  # Rook
            if delta_row != 0 and delta_col != 0:
                return False  # Rook can only move horizontally or vertically

            # Check for obstacles along the path
            if delta_row == 0:  # Horizontal movement
                start, end = min(start_col, dest_col), max(start_col, dest_col)
                for col in range(start + 1, end):
                    if self.board[start_row][col] != '':
                        return False
            else:  # Vertical movement
                start, end = min(start_row, dest_row), max(start_row, dest_row)
                for row in range(start + 1, end):
                    if self.board[row][start_col] != '':
                        return False
            return True

        elif piece in ['♗', '♝']:  # Bishop
            if abs(delta_row) != abs(delta_col):
                return False  # Bishop can only move diagonally

            # Check for obstacles along the path
            row_step = 1 if delta_row > 0 else -1
            col_step = 1 if delta_col > 0 else -1
            row, col = start_row + row_step, start_col + col_step
            while row != dest_row:
                if self.board[row][col] != '':
                    return False
                row += row_step
                col += col_step
            return True

        elif piece in ['♕', '♛']:  # Queen
            
            if abs(delta_row) == abs(delta_col):
                # Check for obstacles along the diagonal path
                row_step = 1 if delta_row > 0 else -1
                col_step = 1 if delta_col > 0 else -1
                row, col = start_row + row_step, start_col + col_step
                while row != dest_row:
                    if self.board[row][col] != '':
                        return False
                    row += row_step
                    col += col_step
                return True
            
            elif delta_row == 0 or delta_col == 0:
                # Check for obstacles along the horizontal or vertical path
                if delta_row == 0:  # Horizontal movement
                    start, end = min(start_col, dest_col), max(start_col, dest_col)
                    for col in range(start + 1, end):
                        if self.board[start_row][col] != '':
                            return False
                else:  # Vertical movement
                    start, end = min(start_row, dest_row), max(start_row, dest_row)
                    for row in range(start+1, end):
                        if self.board[row][start_col] != '':
                            return False
                return True
            else:
                return False  # Invalid movement for the queen

        elif piece in ['♘', '♞']:  # Knight
            # Knights move in an L-shape pattern
            if abs(delta_row) == 2 and abs(delta_col) == 1:
                # Valid vertical L-shape movement
                return True
            elif abs(delta_row) == 1 and abs(delta_col) == 2:
                # Valid horizontal L-shape movement
                return True
            else:
                return False

        elif piece in ['♔', '♚']:  # King
            if abs(delta_row) <= 1 and abs(delta_col) <= 1:
                # Valid king movement (up to 1 square in any direction)
                return True
            else:
                return False

        elif piece == '♙':  # White Pawn
            if delta_col == 0 and delta_row == 1 and self.board[dest_row][dest_col] == '':
                # Pawn moves one step forward
                return True
            elif delta_col == 0 and delta_row == 2 and start_row == 1 and self.board[dest_row][dest_col] == '':
                # Pawn moves two steps forward from the initial position
                return True
            elif abs(delta_col) == 1 and delta_row == 1 and self.is_black(self.board[dest_row][dest_col]):
                # Pawn captures diagonally
                return True
            else:
                return False

        elif piece == '♟':  # Black Pawn
            if delta_col == 0 and delta_row == -1 and self.board[dest_row][dest_col] == '':
                # Pawn moves one step forward
                return True
            elif delta_col == 0 and delta_row == -2 and start_row == 6 and self.board[dest_row][dest_col] == '':
                # Pawn moves two steps forward from the initial position
                return True
            elif abs(delta_col) == 1 and delta_row == -1 and self.is_white(self.board[dest_row][dest_col]):
                # Pawn captures diagonally
                return True
            else:
                return False

        return False  # Invalid move
        
    def draw_board(self):
        for row in range(8):
            for col in range(8):
                x1 = col * self.square_size 
                y1 = row * self.square_size 
                x2 = x1 + self.square_size 
                y2 = y1 + self.square_size 
                
                if (row + col) % 2 == 0:
                    rectangle_id=self.canvas.create_rectangle(x1, y1, x2, y2, fill='#4481A7',width=2)
                else:
                    rectangle_id=self.canvas.create_rectangle(x1, y1, x2, y2, fill='white',width=2)
                    
                # Assign rectangle id to rectangle ids matrix
                self.assign_rectangle_id(row,col,rectangle_id)
                
                # Assign text_id to text_ids8
                text_id=self.canvas.create_text(x1 + self.square_size/2, y1 + self.square_size/2, text=self.board[row][col], font=("Arial", 20))
                self.assign_text_id(row,col,text_id)
                
                self.canvas.tag_bind(rectangle_id, '<Button-1>', self.rectangle_clicked)       
        self.place_piece('♔', 0, 4)
        self.place_piece('♚', 7, 4)
        self.place_piece('♕', 0, 3)
        self.place_piece('♛', 7, 3)
        self.place_piece('♖', 0, 0)
        self.place_piece('♜', 7, 0)
        self.place_piece('♗', 0, 2)
        self.place_piece('♝', 7, 2)
        self.place_piece('♘', 0, 1)
        self.place_piece('♞', 7, 1)
        self.place_piece('♗', 0, 5)
        self.place_piece('♝', 7, 5)
        self.place_piece('♘', 0, 6)
        self.place_piece('♞', 7, 6)
        self.place_piece('♖', 0, 7)
        self.place_piece('♜', 7, 7)
        for i in range(8):
            self.place_piece('♙', 1, i)
            self.place_piece('♟', 6, i)
        
    def rectangle_clicked(self, event):
        rectangle_id = event.widget.find_closest(event.x, event.y)[0]
        row, col = self.find_rectangle_coordinates(rectangle_id)
        clicked_piece = self.board[row][col]

        if self.selected_piece is None:
            # if white or black is selected else nothing
            if clicked_piece != '' and ((self.counter%2!=0 and self.is_black(clicked_piece)) or (self.counter%2==0 and self.is_white(clicked_piece))):
                self.selected_piece = clicked_piece
                self.selected_rectangle = rectangle_id
                self.prow,self.pcol=row,col
                self.canvas.itemconfig(rectangle_id, outline='#7AFFF6', width=2)
                
        elif self.selected_rectangle == rectangle_id:
            # Deselect the previously selected piece
            self.selected_piece = None
            self.prow, self.pcol = None, None
            self.canvas.itemconfig(rectangle_id, outline='', width=2)
                
                
        else:
            if (self.is_black(self.selected_piece) and self.is_black(clicked_piece)) or (self.is_white(self.selected_piece) and self.is_white(clicked_piece)):
                pass
            else:
                if self.move(self.selected_piece,self.prow,self.pcol,row,col) is True:
                    selected_row, selected_col = self.find_rectangle_coordinates(self.selected_rectangle)
                    self.place_piece('', selected_row, selected_col)  # Clear the previous position
                    self.place_piece(self.selected_piece, row, col)  # Move the piece to the new position
                    self.selected_piece = None
                    self.prow,self.pcol=None,None
                    self.canvas.itemconfig(self.selected_rectangle, outline='', width=2)
                    self.counter+=1
                    if self.is_king_under_attack('♔'):
                        messagebox.showinfo("Danger!", "White King in danger!")
                        
                        
                    elif self.is_king_under_attack('♚'):
                        messagebox.showinfo("Danger!", "Black King in danger!")
                    print(self.counter)
                else:
                    print('Invalid move')

    def find_rectangle_coordinates(self, rectangle_id):
        for row in range(8):
            for col in range(8):
                if self.rectangle_ids[row][col] == rectangle_id:
                    return row, col
            
    def place_piece(self, piece, row, col):
        x1 = col * self.square_size
        y1 = row * self.square_size

        self.assign_board(row, col, piece)  # Update the board first

        text_id = self.text_ids[row][col]
        self.canvas.itemconfigure(text_id, text=piece,width=2)  # Update the text of the existing text item

        
    def run(self):
        self.master.mainloop()
        
root = tk.Tk()
chessboard = ChessBoard(root)
chessboard.run()